# EVENT STUDIES
## Quick brush-up

## Event study setup:
Before running the event study, we need to make sure our data is in event time

In [ ]:
* Create event time
    gen event_time= treatment_date - date

: 

## Event studies in Stata:
### Manual event study:

In [ ]:
*** This code snippet runs a manual event study

* Make dummies representing each event_time value
	keep if inrange(event_time, low_bound, upper_bound)
	char event_time[omit] -1  // Chose which value will be omitted by "xi"
	xi i.event_time, pref("i_")

* Run regression of choice:
	global event_dummies i_event_tim_1-i_event_tim_10
	
	regdhf outcome $event_dummies, absorb(fixed effects) vce(cluster id)

* Get coefiecients and SEs
	matrix T=r(table)
	
	capture drop voef* boundH* boundL* se*
	gen coef = 0 if event_time==-1
	gen se = 0 if event_time==-1

* Fill in values
	foreach x of numlist 1/4 6/10 {
		replace coef=T[1,colnumb(T, "i_event_tim_`x'")] if event_time==`x' - 6
		replace se=T[2,colnumb(T, "i_event_tim_`x'")] if event_time==`x' - 6
	}

    gen boundH = coef + 1.96*se
    gen boundL = coef - 1.96*se

    keep if inrange(event_time, lower_bound-1, upper_bound)
    collapse coef boundH boundL, by(event_time)
    tw (rcap boundH boundL event_time, lcolor(black)) (scatter coef event_time, mcolor(black)), ///
        xtitle("") xline(-0.5, lcolor(gray)) legend(off) subtitle("") 

: 